In [18]:
import pandas as pd
import glob
import csv
import json

In [19]:
DATA_DIR = "data/mtg/"
filenames = glob.glob(DATA_DIR + "*.json")

In [20]:
card_template = """
Name: {name}
Colors: {colors}
Manacost: {manacost}
Type: {card_type}
Text: {text}
Set: {set_name} ({set_code})
Release date: {release_date}
"""

def format_card(card, set_data):
    try:
        text = card_template.format(
            name=card["name"],
            colors=", ".join(card["colors"]),
            manacost=card.get("manaCost", "None"),
            card_type=card["type"],
            text=card.get("text", "-"),
            set_code=set_data["code"],
            set_name=set_data["name"],
            release_date=set_data["releaseDate"]
        )

        return pd.Series({
            'text': text,
            'card_id': card["name"],
            'release_date': pd.to_datetime(set_data["releaseDate"]),
            'set_code': set_data["code"]
            
        })
    except:
        print("ERROR")
        print(card["name"], '-', set_data["code"], '\r\n', card)
        raise "boom"

with open(filenames[0]) as f:
    data = json.load(f)["data"]

    for card in data["cards"][5:10]:
        print(format_card(card, data))

text            \nName: Knight of the Hokey Pokey\nColors: W\n...
card_id                                 Knight of the Hokey Pokey
release_date                                  1998-08-11 00:00:00
set_code                                                      UGL
dtype: object
text            \nName: Lexivore\nColors: W\nManacost: {3}{W}\...
card_id                                                  Lexivore
release_date                                  1998-08-11 00:00:00
set_code                                                      UGL
dtype: object
text            \nName: Look at Me, I'm the DCI\nColors: W\nMa...
card_id                                   Look at Me, I'm the DCI
release_date                                  1998-08-11 00:00:00
set_code                                                      UGL
dtype: object
text            \nName: Mesa Chicken\nColors: W\nManacost: {W}...
card_id                                              Mesa Chicken
release_date                      

In [21]:
df = pd.DataFrame(columns=["text", "release_date", "set_code"])

for filename in filenames:
    with open(filename) as f:
        data = json.load(f)["data"]

        print(f'Adding {len(data["cards"])} cards to {data["name"]}')
        
        cards = [format_card(card, data) for card in data["cards"] if "Creature" in card["type"]]
        
        df = pd.concat([df, pd.DataFrame(cards)])

Adding 88 cards to Unglued
Adding 13 cards to Magic Player Rewards 2009
Adding 100 cards to Shadows over Innistrad Promos
Adding 10 cards to Commander's Arsenal Oversized
Adding 0 cards to League Tokens 2012
Adding 18 cards to Wizards Play Network 2024
Adding 15 cards to From the Vault: Relics
Adding 307 cards to Foreign Black Border
Adding 8 cards to Judge Gift Cards 2011
Adding 0 cards to The Brothers' War Art Series
Adding 5 cards to MKM Standard Showdown
Adding 90 cards to Battle for Zendikar Promos
Adding 88 cards to Duel Decks: Jace vs. Vraska
Adding 34 cards to Premium Deck Series: Fire and Lightning
Adding 17 cards to IDW Comics Inserts
Adding 9 cards to Born of the Gods Promos
Adding 188 cards to Crimson Vow Commander
Adding 358 cards to Theros Beyond Death
Adding 3 cards to Tenth Edition Promos
Adding 4 cards to Ponies: The Galloping
Adding 350 cards to Tempest


/tmp/ipykernel_343/1966839644.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame(cards)])


Adding 5 cards to Judge Gift Cards 2008
Adding 0 cards to Modern Horizons 3 Substitute Cards
Adding 143 cards to Exodus
Adding 81 cards to Foundations Promos
Adding 0 cards to Strixhaven: School of Mages Minigames
Adding 5 cards to Magic Premiere Shop 2006
Adding 1 cards to M15 Prerelease Challenge
Adding 1 cards to MagicFest 2024
Adding 309 cards to Assassin's Creed
Adding 113 cards to Duels of the Planeswalkers
Adding 92 cards to Arabian Nights
Adding 198 cards to Magic Online Theme Decks
Adding 730 cards to Foundations
Adding 9 cards to Arena League 2006
Adding 2 cards to Judge Gift Cards 2002
Adding 210 cards to Conspiracy
Adding 384 cards to Double Masters
Adding 9 cards to Friday Night Magic 2017
Adding 351 cards to Classic Sixth Edition
Adding 8 cards to Judge Gift Cards 2019
Adding 2 cards to Duels of the Planeswalkers 2015 Promos
Adding 592 cards to Tales of Middle-earth Commander
Adding 4 cards to Tarkir Dragonfury
Adding 223 cards to Eldritch Moon
Adding 302 cards to Collect

In [22]:
df.head()

,text,release_date,set_code,card_id
0,\nName: Knight of the Hokey Pokey\nColors: W\n...,1998-08-11,UGL,Knight of the Hokey Pokey
1,\nName: Lexivore\nColors: W\nManacost: {3}{W}\...,1998-08-11,UGL,Lexivore
2,\nName: Mesa Chicken\nColors: W\nManacost: {W}...,1998-08-11,UGL,Mesa Chicken
3,\nName: Miss Demeanor\nColors: W\nManacost: {3...,1998-08-11,UGL,Miss Demeanor
4,\nName: Chicken à la King\nColors: U\nManacost...,1998-08-11,UGL,Chicken à la King


In [25]:
unique_cards = df.sort_values(by=["release_date"]).drop_duplicates("card_id", keep="first")

In [26]:
unique_cards.to_csv("data/cards.csv", index=False)